# Exercise 16

cf. [Boston data set documentation](https://islp.readthedocs.io/en/latest/datasets/Boston.html)

In [60]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from ISLP import load_data
from ISLP import confusion_table
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from ISLP.models import ModelSpec as MS

Boston = load_data("Boston")
Boston.shape, Boston.columns

((506, 13),
 Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
        'ptratio', 'lstat', 'medv'],
       dtype='object'))

In [61]:
criteria = Boston.crim > Boston.crim.median()
crim01 = np.array(["0"]*Boston.shape[0])
crim01[criteria] = "1"
Boston["crim01"] = crim01

Boston.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000


## Logistic regression

In [62]:
(X_train, X_test, y_train, y_test) = train_test_split(Boston.drop(columns=["crim","crim01"]), crim01, test_size=0.5, random_state=0)

# For logistic regression, we need to add the intercept (if you use MS it will add it for you, but I want to use the same sets for all methods)
X_train_ = X_train
X_test_ = X_test
X_train_["intercept"] = 1
X_test_["intercept"] = 1

glm = sm.GLM(y_train == "1",X_train_,family=sm.families.Binomial())
results = glm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  253
Model:                            GLM   Df Residuals:                      240
Model Family:                Binomial   Df Model:                           12
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -50.369
Date:                Tue, 12 Sep 2023   Deviance:                       100.74
Time:                        10:04:17   Pearson chi2:                     666.
No. Iterations:                     9   Pseudo R-squ. (CS):             0.6270
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
zn            -0.0713      0.044     -1.637      0.102      -0.157       0.014
indus         -0.1543      0.075     -2.070      0.038      -0.300      -0.008
chas           0.9984      1.010      0.988      0.323      -0.982       2.979
nox           53.2761     11.503      4.631      0.000      30.730      75.822
rm            -0.3268      1.014     -0.322      0.747      -2.315       1.661
age            0.0303      0.017      1.816      0.069      -0.002       0.063
dis            0.8007      0.340      2.357      0.018       0.135       1.466
rad            0.6037      0.226      2.671      0.008       0.161       1.047
tax           -0.0033      0.004     -0.811      0.417      -0.011       0.005
ptratio        0.5476      0.200      2.737      0.006       0.156       0.940
lstat          0.2114      0.076      2.773      0.006       0.062       0.361
medv           0.2765      0.116      2.377      0.017       0.049       0.505
intercept    -51.3460     10.827     -4.742      0.000     -72.566     -30.126
==============================================================================
"""

From the summary we can see that `zn`, `chas`, `rm` and `tax` are not statistically significant to predict the response.
So we will try again by removing these parameters.

In [63]:
(X_train, X_test, y_train, y_test) = train_test_split(Boston.drop(columns=["crim","crim01","zn","chas","rm","tax"]), crim01, test_size=0.5, random_state=0)
X_train_ = X_train
X_test_ = X_test
X_train_["intercept"] = 1
X_test_["intercept"] = 1

glm = sm.GLM(y_train == "1",X_train_,family=sm.families.Binomial())
results = glm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  253
Model:                            GLM   Df Residuals:                      244
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -54.190
Date:                Tue, 12 Sep 2023   Deviance:                       108.38
Time:                        10:04:17   Pearson chi2:                     256.
No. Iterations:                     9   Pseudo R-squ. (CS):             0.6156
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
indus         -0.1912      0.068     -2.822      0.005      -0.324      -0.058
nox           51.1504     11.473      4.458      0.000      28.664      73.637
age            0.0274      0.014      1.963      0.050    4.48e-05       0.055
dis            0.5948      0.254      2.342      0.019       0.097       1.093
rad            0.4892      0.162      3.027      0.002       0.172       0.806
ptratio        0.5354      0.172      3.120      0.002       0.199       0.872
lstat          0.2320      0.072      3.205      0.001       0.090       0.374
medv           0.2486      0.070      3.535      0.000       0.111       0.386
intercept    -50.9489     10.325     -4.935      0.000     -71.185     -30.713
==============================================================================
"""

Now everything looks statistically significant, we can predict on the holdout data and compute the test error.

In [64]:
probs = results.predict(exog=X_test_)
labels = np.array(["0"]*X_test_.shape[0])
labels[probs>0.5] = "1"
confusion_table(labels, y_test)

Truth,0,1
Predicted,,
0,111,23
1,10,109


In [65]:
np.mean(labels != y_test)

0.13043478260869565

The test error for logistic regression is 13%

## LDA

In [66]:
(X_train, X_test, y_train, y_test) = train_test_split(Boston.drop(columns=["crim","crim01","zn","chas","rm","tax"]), crim01, test_size=0.5, random_state=0)

lda = LDA(store_covariance=True)
lda.fit(X_train, y_train)
lda_pred = lda.predict(X_test)
confusion_table(lda_pred, y_test)

Truth,0,1
Predicted,,
0,120,39
1,1,93


In [67]:
np.mean(lda_pred != y_test)

0.15810276679841898

LDA has an overall test error of 15.8%.

But looking in details, type I error are much better: 1/(120 +1) = 0.8% compare to logistic regression that's an order of magnitude higher 10/(111+10) = 8%

So we could use that as a strategy where we only look at Non-null hypothesis (aka when the crim is above the median).

## QDA

In [68]:
(X_train, X_test, y_train, y_test) = train_test_split(Boston.drop(columns=["crim","crim01","zn","chas","rm","tax"]), crim01, test_size=0.5, random_state=0)

qda = QDA(store_covariance=True)
qda.fit(X_train, y_train)
qda_pred = qda.predict(X_test)
confusion_table(qda_pred, y_test)

Truth,0,1
Predicted,,
0,120,37
1,1,95


In [69]:
np.mean(qda_pred != y_test)

0.15019762845849802

The error test rate for QDA is 15%.
As for LDA, type I error is very low, but unlike it, type II error is a bit lower. So overall it might be a better strategy to use than LDA.

## Naive Bayes

In [70]:
(X_train, X_test, y_train, y_test) = train_test_split(Boston.drop(columns=["crim","crim01","zn","chas","rm","tax"]), crim01, test_size=0.5, random_state=0)

NB = GaussianNB()
NB.fit(X_train, y_train)
NB_pred = NB.predict(X_test)
confusion_table(NB_pred, y_test)

Truth,0,1
Predicted,,
0,110,27
1,11,105


In [71]:
np.mean(NB_pred != y_test)

0.15019762845849802

For Naive Bayes, the overall test error is 15%, so the same as for QDA.

Now, depending on what we are mostly interested (i.e. the null/non-null hypothesis), one might choose one or the another.

In this case, I would imagine that people are more worried about a prediction of low crime rate and buying a house with high crime rate. So we are interested in reducing the error when we predict the null hypothesis (crim01 = 0), but we end up with a true class non null (crim01 = 1). This is a type II error (a.k.a. a false negative), when we said crim rate will be low but it will be high instead, and that's the top right corner of the confusion table.

So back to our method comparison, if we want to reduce the type II error, we would choose Naive Bayes over QDA.

## KNN

In [72]:
for K in range(1,6):
    knn = KNeighborsClassifier(n_neighbors=K)
    knn_pred = knn.fit(X_train.values, y_train).predict(X_test.values)
    C = confusion_table(knn_pred, y_test)
    templ = ("K={0:d}: # predicted with higher crim rate: {1:>2}," +
             " # with actual high crime rate {2:d}, test error rate {3:.1%}")
    pred = C.loc["0"].sum()
    high_crim_rate = C.loc["0", "0"]
    print(templ.format(K, pred, high_crim_rate, 1-(high_crim_rate/pred)))

K=1: # predicted with higher crim rate: 129, # with actual high crime rate 105, test error rate 18.6%
K=2: # predicted with higher crim rate: 151, # with actual high crime rate 117, test error rate 22.5%
K=3: # predicted with higher crim rate: 137, # with actual high crime rate 112, test error rate 18.2%
K=4: # predicted with higher crim rate: 152, # with actual high crime rate 114, test error rate 25.0%
K=5: # predicted with higher crim rate: 137, # with actual high crime rate 108, test error rate 21.2%


In the case of the KNN, the lowest error rate for the case we are interested in (when we predict low rate crime) is for K=1 and is 18.6%. Naive Bayes is still a better option.